In [1]:
from astropy.coordinates import SkyCoord
from astropy.table import Table
from IPython.display import display
import csv

In [2]:
ALL_FILTERS = ('B', 'V', 'R', 'I')

targets = {
    'HD38451': ALL_FILTERS,
    '27Cephei': ALL_FILTERS,
}

In [3]:
MAX_FLAG_THRESHOLD = 0.25
DEFAULT_EXPOSURES = range(5, 60, 5)
MIN_EXPOSURES = 5

In [4]:
with open('data/schedule_pirate.csv', 'w') as csv_file:
    csv_writer = csv.writer(csv_file)
    csv_writer.writerow((
        'Name', 'RA', 'Dec', 'is timed', 'start time', 'end time',
        'Priority', 'Atoms', 'Binning', 'repeats', 'Max uses', 'Alt limit'
    ))
    for target, filters in targets.items():
        _filters = {}
        for f in filters:
            _filters[f] = []
            for e in DEFAULT_EXPOSURES:
                 _filters[f].append(float(e))
        filters = _filters
        
        try:
            photometry = Table.read(f'data/{target}.ecsv')
            photometry['flagged'] = photometry['FLAGS'] != 0
            for band, group in photometry.to_pandas().groupby('band'):
                pt = group[['exposure', 'flagged']].pivot_table(
                    columns='flagged',
                    index=['exposure'],
                    aggfunc=lambda x: len(x),
                    fill_value=0,
                )
                pt['total'] = pt[True] + pt[False]
                pt['frac'] = pt[True] / pt['total']
                
                flagged_exposures = pt[(pt['total'] >= MIN_EXPOSURES) & (pt['frac'] >= MAX_FLAG_THRESHOLD)].index
                if len(flagged_exposures) > 0:
                    min_flagged_exposure = pt[(pt['total'] >= MIN_EXPOSURES) & (pt['frac'] >= MAX_FLAG_THRESHOLD)].index.min()
                    filters[band] = [e for e in filters[band] if e < min_flagged_exposure]
                    filters[band] = [max(filters[band])]
        except FileNotFoundError:
            pass

        coords = SkyCoord.from_name(target, parse=True)
        atoms = []
        for f, es in filters.items():
            for e in es[len(es)-3:]:
                atoms.append(f'{f}:{e}')
        atoms = ';'.join(atoms)

        csv_writer.writerow([
            target, coords.ra.to_string(alwayssign=True, sep=':'),
            coords.dec.to_string(alwayssign=True, sep=':'), 'FALSE', '',
            '', '500', atoms, '1', '1', '3', '20',
        ])